In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.sonic.net/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xzf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
df = spark.read.format('csv').options(inferSchema=True, header=True).load('/content/drive/MyDrive/Colab Notebooks/Trees/college_data.csv')

In [ ]:
df.printSchema()

root
 |-- private: string (nullable = true)
 |-- apps: integer (nullable = true)
 |-- accept: integer (nullable = true)
 |-- enroll: integer (nullable = true)
 |-- top10perc: integer (nullable = true)
 |-- top25perc: integer (nullable = true)
 |-- f_undergrad: integer (nullable = true)
 |-- p_undergrad: integer (nullable = true)
 |-- outstate: integer (nullable = true)
 |-- room_board: integer (nullable = true)
 |-- books: integer (nullable = true)
 |-- personal: integer (nullable = true)
 |-- phd: integer (nullable = true)
 |-- terminal: integer (nullable = true)
 |-- s_f_ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- expend: integer (nullable = true)
 |-- grad_rate: integer (nullable = true)



In [ ]:
df.describe().show()

+-------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+------------------+
|summary|private|              apps|            accept|          enroll|         top10perc|         top25perc|      f_undergrad|      p_undergrad|          outstate|        room_board|             books|          personal|               phd|          terminal|         s_f_ratio|       perc_alumni|          expend|         grad_rate|
+-------+-------+------------------+------------------+----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+---------------

In [ ]:
df.columns

['private',
 'apps',
 'accept',
 'enroll',
 'top10perc',
 'top25perc',
 'f_undergrad',
 'p_undergrad',
 'outstate',
 'room_board',
 'books',
 'personal',
 'phd',
 'terminal',
 's_f_ratio',
 'perc_alumni',
 'expend',
 'grad_rate']

In [ ]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
assembler = VectorAssembler(
    inputCols=[
        'apps',
        'accept',
        'enroll',
        'top10perc',
        'top25perc',
        'f_undergrad',
        'p_undergrad',
        'outstate',
        'room_board',
         'books',
         'personal',
         'phd',
         'terminal',
         's_f_ratio',
         'perc_alumni',
         'expend',
         'grad_rate'], 
    outputCol='features')
stridx = StringIndexer(inputCol='private', outputCol='label')
final_data = assembler.transform(df)
final_data = stridx.fit(final_data).transform(final_data)

In [ ]:
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier,
                                       DecisionTreeClassifier)

In [ ]:
train, test = final_data.randomSplit([0.7, 0.3])

In [ ]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees = 100)
gbt = GBTClassifier()

In [ ]:
dtc_model = dtc.fit(train)
rfc_model = rfc.fit(train)
gbt_model = gbt.fit(train)

In [ ]:
dtc_preds = dtc_model.transform(test)
rfc_preds = rfc_model.transform(test)
gbt_preds = gbt_model.transform(test)

In [ ]:
dtc_preds.show()

+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+-----+-------------+--------------------+----------+
|private|apps|accept|enroll|top10perc|top25perc|f_undergrad|p_undergrad|outstate|room_board|books|personal|phd|terminal|s_f_ratio|perc_alumni|expend|grad_rate|            features|label|rawPrediction|         probability|prediction|
+-------+----+------+------+---------+---------+-----------+-----------+--------+----------+-----+--------+---+--------+---------+-----------+------+---------+--------------------+-----+-------------+--------------------+----------+
|     No| 529|   481|   243|       22|       47|       1206|        134|    4860|      3122|  600|     650|103|      88|     17.4|         16|  6415|       43|[529.0,481.0,243....|  1.0|    [0.0,4.0]|           [0.0,1.0]|       1.0|
|     No| 535|   502|   223|        6|       20|        959|        

In [ ]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees = 100)
gbt = GBTClassifier()

In [ ]:
dtc_model = dtc.fit(train)
rfc_model = rfc.fit(train)
gbt_model = gbt.fit(train)

In [ ]:
dtc_preds = dtc_model.transform(test)
rfc_preds = rfc_model.transform(test)
gbt_preds = gbt_model.transform(test)

In [ ]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[121,122,123...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[122,123,148...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[125,126,127...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[153,154,155...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[153,154,155...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[181,182,183...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[234,235,237...|   [31.0,0.0]|  [1.0,0.0]|       0.0|
|  1.0|(69

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

In [ ]:
print(f'DTC: {evaluator.evaluate(dtc_preds)}') 
print(f'RFC: {evaluator.evaluate(rfc_preds)}')
print(f'GBT: {evaluator.evaluate(gbt_preds)}')

DTC: 0.9196428571428571
RFC: 0.9375
GBT: 0.9151785714285714
